<h1>GLOBAL TERRORIST ANALYSIS</h1>

In [4]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')
import folium
from folium.plugins import HeatMap
from folium import FeatureGroup

In [7]:
# Load the dataset with the correct encoding
df = pd.read_csv('dataset\\Global_terrorist.csv', encoding='ISO-8859-1')

In [8]:
df

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,...,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,2017,12,31,NaN,0,NaN,200,Syria,10,...,NaN,"""Putin's 'victory' in Syria has turned into a ...","""Two Russian soldiers killed at Hmeymim base i...","""Two Russian servicemen killed in Syria mortar...",START Primary Collection,-9,-9,1,1,NaN
181688,201712310030,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Maguindanao clashes trap tribe members,"" Phil...",NaN,NaN,START Primary Collection,0,0,0,0,NaN
181689,201712310031,2017,12,31,NaN,0,NaN,92,India,6,...,NaN,"""Trader escapes grenade attack in Imphal,"" Bus...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN


<h1>Filtering the data frame</h1>

In [9]:
filter1 = df[df['region_txt']== 'Middle East & North Africa']

In [10]:
filter2 = filter1[filter1['iyear'] >= 2000]

In [11]:
filter3 = filter2[filter2['attacktype1_txt'] == 'Bombing/Explosion']

In [12]:
filter4 = filter3[filter3['nkill'] >= 10]

In [13]:
aggregated_df = filter4.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) / (
    aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
heat_data = aggregated_df[['latitude', 'longitude','normalized_weight']].values.tolist()
m = folium.Map(location = [30,40], zoom_start=4)
HeatMap(heat_data).add_to(m)
m

<h1> REGIONAL ANALYSIS </h1>
<h3> Heatmap of Terrorist Attack Intensity by Region and Attack Type in South Asia </h3>

In [14]:
region_name = 'South Asia' 
df_region = df[df['region_txt'] == region_name]

df_region = df_region.dropna(subset=['latitude', 'longitude', 'attacktype1_txt'])

m = folium.Map(location=[df_region['latitude'].mean(), df_region['longitude'].mean()], zoom_start=5)

attack_types = df_region['attacktype1_txt'].unique()
attack_type_groups = {}

for attack_type in attack_types:
    df_attack = df_region[df_region['attacktype1_txt'] == attack_type]
    heat_data = [[row['latitude'], row['longitude']] for index, row in df_attack.iterrows()]
    
    fg = FeatureGroup(name=attack_type)
    HeatMap(heat_data).add_to(fg)
    attack_type_groups[attack_type] = fg
    fg.add_to(m)

folium.LayerControl().add_to(m)

m.save('regional_attack_heatmap.html')

m

<h1> CRIME TYPE ANALYSIS</h1>

<h3> Heatmap of Armed Assault Incidents Across Sub-Saharan Africa </h3>

In [15]:
africa_data = df[df['region_txt'] == "Sub-Saharan Africa"]
africa_armed_assault = africa_data[africa_data['attacktype1_txt'] == "Armed Assault"]
africa_armed_assault = africa_armed_assault.dropna(subset=["latitude", "longitude"])

In [16]:
map_africa_armed_assault = folium.Map(location=[8.7832, 34.5085], zoom_start=4, tiles="OpenStreetMap")

In [17]:
heat_data_africa = africa_armed_assault[['latitude', 'longitude']].values.tolist()
HeatMap(heat_data_africa, radius=6, blur=5, max_zoom=10, min_opacity=0.4).add_to(map_africa_armed_assault)
map_africa_armed_assault

<h1> CASUALTY ANALYSIS </h1>

<h3> Regional Heatmap of Terrorist Group Casualties in the Middle East & North Africa</h3>

In [18]:
df['nkill'] = df['nkill'].fillna(0)
df['nwound'] = df['nwound'].fillna(0)
df['casualties'] = df['nkill'] + df['nwound']

region_name = 'Middle East & North Africa'
df_region = df[df['region_txt'] == region_name]


df_region = df_region.dropna(subset=['latitude', 'longitude', 'gname'])
df_region = df_region[df_region['gname'] != 'Unknown']

m = folium.Map(location=[df_region['latitude'].mean(), df_region['longitude'].mean()], zoom_start=5)

groups = df_region['gname'].value_counts().head(6).index  # Top 6 groups for clarity

for group in groups:
    df_group = df_region[df_region['gname'] == group]
    
    # Prepare heat data with casualties as weight
    heat_data = [[row['latitude'], row['longitude'], row['casualties']] for index, row in df_group.iterrows()]
    
    fg = FeatureGroup(name=group)
    HeatMap(
        heat_data,
        radius=6,
        blur=4,
        min_opacity=0.3,
        max_zoom=10
    ).add_to(fg)
    
    fg.add_to(m)

folium.LayerControl().add_to(m)

m.save("group_casualty_heatmap.html")
m

<h1> DEMOGRAPHIC ANALYSIS</h1>

<h3> Geographic Distribution of Civilian-Targeted Terrorist Attacks in South Asia</h3>

In [19]:
df_region = df[
    (df['region_txt'] == 'South Asia') &  # FOCUS REGION
    (df['targtype1_txt'] == 'Private Citizens & Property') &
    (df['nkill'] >= 1) &
    (df['weaptype1_txt'].notna()) &
    (df['latitude'].notna()) &
    (df['longitude'].notna())
]

heat_data = df_region[['latitude', 'longitude', 'nkill']].values.tolist()

map_region = folium.Map(location=[25, 80], zoom_start=5, tiles='OpenStreetMap')

HeatMap(
    heat_data,
    radius=6,
    blur=4,
    max_zoom=10,
    min_opacity=0.4
).add_to(map_region)

map_region.save("civilian_deaths_south_asia.html")
map_region

<h1> TARGET ANALYSIS </h1>

<h3> Geographic Distribution of Terrorist Attacks Across Target Types in the Middle East & North Africa </h3>

In [20]:
df['nkill'] = df['nkill'].fillna(0)
df['nwound'] = df['nwound'].fillna(0)
df['casualties'] = df['nkill'] + df['nwound']

region_name = 'Middle East & North Africa'
df_region = df[df['region_txt'] == region_name]
df_region = df_region.dropna(subset=['latitude', 'longitude', 'targtype1_txt'])
df_region = df_region[df_region['targtype1_txt'] != 'Unknown']

In [21]:
m = folium.Map(location=[df_region['latitude'].mean(), df_region['longitude'].mean()], zoom_start=5)

target_types = df_region['targtype1_txt'].unique()

for target in target_types:
    df_target = df_region[df_region['targtype1_txt'] == target]
    heat_data = [[row['latitude'], row['longitude'], row['casualties']] for index, row in df_target.iterrows()]
    fg = FeatureGroup(name=target)
    
    HeatMap(
        heat_data,
        radius=6,
        blur=5,
        min_opacity=0.3,
        max_zoom=10
    ).add_to(fg)
    fg.add_to(m)

folium.LayerControl().add_to(m)

m.save("mena_target_type_heatmap.html")

m